In [1]:
import os
import torch

os.chdir('/home/robsyc/Desktop/thesis/MB-VAE-DTI/')
print(os.listdir('.'))
print(os.getcwd())

print(torch.cuda.is_available())
torch.cuda.empty_cache()

['.git', 'utils', 'data', 'bmfm_sm', 'scripts', '.gitignore', 'README.md', 'ESPF', 'notebooks', 'hpc.pbs', 'requirements.txt']
/home/robsyc/Desktop/thesis/MB-VAE-DTI
False


/home/robsyc/miniconda3/envs/mbvae_env/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
from utils.modelBuilding import ResidualMLP, DrugTargetTree
import torch

model = ResidualMLP(
    input_dim = 512,
    hidden_dim = 256,
    output_dim = 64,
    depth = 0,
    dropout_prob = 0.1,
    activation = 'SiLU',
)
x = torch.randn(32, 512)
print("ResidualMLP output:", model(x).shape, "\n")

model = DrugTargetTree(
    drug_dims = [512, 512, 768],
    target_dims = [768],
    hidden_dim = 256,
    latent_dim = 512,
    depth = 2,
    dropout_prob = 0.1,
    variational = False
)
drugs = [torch.randn(32, 512), torch.randn(32, 512), torch.randn(32, 768)]
targets = [torch.randn(32, 768)]

output = model(drugs, targets, return_attn=True)
print("DrugTargetTree output:", output.predictions.shape)
print("Attention weights:", output.attention_weights, "\n")

model = DrugTargetTree(
    drug_dims = [512, 512, 768],
    target_dims = [768],
    hidden_dim = 256,
    latent_dim = 256,
    depth = 2,
    dropout_prob = 0.1,
    variational = True
)

output = model(drugs, targets, return_attn=True, compute_kl_loss=True)
print("DrugTargetTree output:", output.predictions.shape)
print("Attention weights:", output.attention_weights)
print("KL loss:", output.kl_loss)

ResidualMLP output: torch.Size([32, 64]) 

Number of parameters
 - Drug Leafs: 4,010,496
 - Target Leafs: 1,380,864
 - Attention: 513
 - Total: 5,391,873
DrugTargetTree output: torch.Size([32])
Attention weights: (tensor([0.3503, 0.2957, 0.3540], grad_fn=<SqueezeBackward0>), None) 

Number of parameters
 - Drug Leafs: 4,010,496
 - Target Leafs: 1,380,864
 - Attention: 513
 - Total: 5,391,873
DrugTargetTree output: torch.Size([32])
Attention weights: (tensor([0.3279, 0.3562, 0.3159], grad_fn=<SqueezeBackward0>), None)
KL loss: tensor(11.2161, grad_fn=<AddBackward0>)


In [9]:
model

DrugTargetTree(
  (drug_leafs): ModuleList(
    (0-1): 2 x ResidualMLP(
      (activation): SiLU()
      (input2hidden): Sequential(
        (0): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (1): Linear(in_features=512, out_features=256, bias=False)
        (2): SiLU()
        (3): Dropout(p=0.1, inplace=False)
      )
      (hidden_layers): ModuleList(
        (0-1): 2 x Sequential(
          (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (1): Linear(in_features=256, out_features=1024, bias=False)
          (2): SiLU()
          (3): Dropout(p=0.1, inplace=False)
          (4): Linear(in_features=1024, out_features=256, bias=False)
        )
      )
      (layer_norms): ModuleList(
        (0-1): 2 x LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      )
      (layer_scales): ParameterList(
          (0): Parameter containing: [torch.float32 of size 256]
          (1): Parameter containing: [torch.float32 of size 256]
      )
      (hidden

---
---

# Load Example Data

In [5]:
import h5torch

dataset = h5torch.Dataset(
    "./data/dataset/DAVIS.h5t",
    sampling="coo",
    subset=("unstructured/split_cold", "valid"),
    in_memory=True,
)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=12, shuffle=True)
batch = next(iter(dataloader))
batch

{'central': tensor([5.0000, 5.0000, 5.4437, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000, 5.0000,
         5.0000, 5.0000, 5.0000]),
 '0/Drug_ID': ['9926054',
  '9926054',
  '216239',
  '11717001',
  '216239',
  '123631',
  '123631',
  '9926791',
  '156422',
  '216239',
  '9926054',
  '9926791'],
 '0/Drug_SMILES': ['Cc1ccc2nc(NCCN)c3ncc(C)n3c2c1.Cl',
  'Cc1ccc2nc(NCCN)c3ncc(C)n3c2c1.Cl',
  'CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)c3)cc2)ccn1',
  'OCCn1cc(-c2ccc3c(c2)CCC3=NO)c(-c2ccncc2)n1',
  'CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)c3)cc2)ccn1',
  'COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1',
  'COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1',
  'CC1CCN(C(=O)CC#N)CC1N(C)c1ncnc2[nH]ccc12',
  'Cc1ccc(-n2nc(C(C)(C)C)cc2NC(=O)Nc2ccc(OCCN3CCOCC3)c3ccccc23)cc1',
  'CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)c3)cc2)ccn1',
  'Cc1ccc2nc(NCCN)c3ncc(C)n3c2c1.Cl',
  'CC1CCN(C(=O)CC#N)CC1N(C)c1ncnc2[nH]ccc12'],
 '0/Drug_emb_graph': tensor([[-0.0104, -0.0124, -0.0965,  ..., -0.0641, -0

In [6]:
print("Molecule drugs")
for key in ["0/Drug_fp", '0/Drug_emb_graph', '0/Drug_emb_image', '0/Drug_emb_text']:
    print("- ", key[2:], batch[key].shape)

print("\nProtein targets")
for key in ['1/Target_fp', '1/Target_emb_ESM', '1/Target_emb_T5', '1/Target_emb_DNA']:
    print("- ", key[2:], batch[key].shape)

print("\nInteraction value: ", batch["central"].shape)

Molecule drugs
-  Drug_fp torch.Size([12, 2048])
-  Drug_emb_graph torch.Size([12, 512])
-  Drug_emb_image torch.Size([12, 512])
-  Drug_emb_text torch.Size([12, 768])

Protein targets
-  Target_fp torch.Size([12, 4170])
-  Target_emb_ESM torch.Size([12, 1280])
-  Target_emb_T5 torch.Size([12, 1024])
-  Target_emb_DNA torch.Size([12, 1024])

Interaction value:  torch.Size([12])


In [7]:
from torch.utils.data import DataLoader, Dataset
import h5torch

def get_dataset(split_type, split_name):
    return h5torch.Dataset(
        "./data/dataset/DAVIS.h5t",
        sampling="coo",
        subset=(f"unstructured/{split_type}", split_name),
        in_memory=True,
    )

class CustomH5Dataset(Dataset):
    def __init__(self, h5_dataset, inputs_0, inputs_1):
        self.dataset = h5_dataset
        self.inputs_0 = inputs_0
        self.inputs_1 = inputs_1

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        batch = self.dataset[idx]
        # Inputs for branch 0 (drug)
        x0 = [batch[key] for key in self.inputs_0]
        # Inputs for branch 1 (target)
        x1 = [batch[key] for key in self.inputs_1]
        # Interaction value
        y = batch['central']
        return x0, x1, y

config = {
        'inputs_0': ['0/Drug_fp'],
        'inputs_1': ['1/Target_fp'],
        'model_type': 'plain',  # 'plain' or 'variational'
}
batch_size = 16

train_dataset = get_dataset("split_rand", 'train')
valid_dataset = get_dataset("split_rand", 'valid')
test_dataset = get_dataset("split_rand", 'test')

# Create DataLoaders
train_loader = DataLoader(
    CustomH5Dataset(train_dataset, config['inputs_0'], config['inputs_1']),
    batch_size=batch_size,
    shuffle=True
)
valid_loader = DataLoader(
    CustomH5Dataset(valid_dataset, config['inputs_0'], config['inputs_1']),
    batch_size=batch_size,
    shuffle=False
)
test_loader = DataLoader(
    CustomH5Dataset(test_dataset, config['inputs_0'], config['inputs_1']),
    batch_size=batch_size,
    shuffle=False
)

In [1]:
import os
import torch
os.chdir('/home/robsyc/Desktop/thesis/MB-VAE-DTI/')
from utils.modelTraining import train_and_evaluate

CONFIGS = {
    'single_view_fp_plain': {
        'inputs_0': ['0/Drug_fp'],
        'inputs_1': ['1/Target_fp'],
        'model_type': 'plain',
        'hyperparamter_grid': {
            'learning_rate': 0.0005,
            'batch_size': 64,
            'depth': 1,
            'hidden_dim': 64,
            'latent_dim': 128,
            'dropout_prob': 0.1,
        }
    },
}

config = CONFIGS['single_view_fp_plain']
hyperparams = config['hyperparamter_grid']
split_type = 'split_rand'

for exp_name, config in CONFIGS.items():
    print(f"\nRunning experiment: {exp_name}\nParams: {hyperparams}\n")
    config = CONFIGS[exp_name]
    hyperparams = config['hyperparamter_grid']
    best_valid_loss, avg_test_loss, num_trainable_params, predictions = train_and_evaluate(
        config=config,
        split_type="split_cold",
        **hyperparams
    )
    print(f"Experiment: {exp_name}, Best Valid Loss: {best_valid_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

/home/robsyc/miniconda3/envs/mbvae_env/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0



Running experiment: single_view_fp_plain
Params: {'learning_rate': 0.0005, 'batch_size': 64, 'depth': 1, 'hidden_dim': 64, 'latent_dim': 128, 'dropout_prob': 0.1}

Number of parameters
 - Drug Leafs: 176,576
 - Target Leafs: 316,628
 - Attention: 0
 - Total: 493,204
Epoch [1], Train Loss: 1.6669, Valid Loss: 0.5019
Epoch [2], Train Loss: 0.5312, Valid Loss: 0.3667
Epoch [3], Train Loss: 0.4514, Valid Loss: 0.3346
Epoch [4], Train Loss: 0.3956, Valid Loss: 0.5152
Epoch [5], Train Loss: 0.3492, Valid Loss: 0.5295
Epoch [6], Train Loss: 0.3305, Valid Loss: 0.3788
Epoch [7], Train Loss: 0.3027, Valid Loss: 0.3933
Epoch [8], Train Loss: 0.2810, Valid Loss: 0.4125
Epoch [9], Train Loss: 0.2633, Valid Loss: 0.3756
Epoch [10], Train Loss: 0.2549, Valid Loss: 0.3524
Epoch [11], Train Loss: 0.2412, Valid Loss: 0.3504
Epoch [12], Train Loss: 0.2271, Valid Loss: 0.3511
Epoch [13], Train Loss: 0.2226, Valid Loss: 0.4017
Epoch [14], Train Loss: 0.2108, Valid Loss: 0.3403
Epoch [15], Train Loss: 0.2